# Assignment 01: Supervised learning, Linear models, and Loss functions

In this assignment, you're going to write your own methods to fit a linear model using either an OLS or Huber loss function.  

## Data set

We will examine some data representing the miles-per-gallon of 398 cars given other variables describing them:

1. mpg: continuous. The miles-per-gallon of the car.
2. cylinders: multi-valued discrete. Number of cylinders.
3. displacement: continuous. Engine displacement of the car.
4. horsepower: continuous. Total horsepower of the car.
5. weight: continuous. Weight in lbs.
6. acceleration: continuous. Acceleration 0-60mph in seconds.
9. car name: string (unique for each instance)

## Follow These Steps Before Submitting
Once you are finished, ensure to complete the following steps.

1.  Restart your kernel by clicking 'Kernel' > 'Restart & Run All'.

2.  Fix any errors which result from this.

3.  Repeat steps 1. and 2. until your notebook runs without errors.

4.  Submit your completed notebook to OWL by the deadline.


## Preliminaries

In [ ]:
# Import all the necessary packages:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss
import scipy.optimize as so
from sklearn import linear_model

%matplotlib inline

In [ ]:
# Uncomment if using Google Colab or Kaggle Kernels.
# Imports the data using gdown.
!gdown https://drive.google.com/uc?id=1PtY3ne37XA8Jk_cAf0Cd7JSRvEU8KDbp


## Part 1
### Question 1.1:


Read the `car_data.csv` file as a `pandas.DataFrame` and show its descriptive statistics.  Investigate the relationship between the cars' weight and their mpg by plotting a scatter plot of the `weight` (x axis) and `mpg` columns (y axis). Add an `alpha`(transparency of the plotted dots) in case some data are overlapping. Remember to label your axes.

In [ ]:
df = pd.read_csv('car_data.csv')

print(df.describe())
print("\ndata shape:", df.shape)
print("\nfirst few rows:")
print(df.head())

plt.figure(figsize=(10, 6))
plt.scatter(df['weight'], df['MPG'], alpha=0.5)
plt.xlabel('weight (lbs)')
plt.ylabel('mpg (miles per gallon)')
plt.title('relationship between car weight and mpg')
plt.grid(True, alpha=0.3)
plt.show()

**Written answer: What do you see here? Discuss your findings**
I can observe a strong negative relationship between car weight and mpg. As the weight of the car increases, the mpg decreases. This makes sense because heavy cars need more energy to move, making fuel efficiency lower. The relationship is linear and lacks clear outliers.

### Question 1.2:

Recall that the linear model, we obtain predictions by computing

$$ \hat{\mathbf{y}} = \mathbf{X} \hat{\beta} $$

Here, $\mathbf{X}$ is a design matrix which includes a column of ones, $\hat{\beta}$ are coefficients, and $\hat{\mathbf{y}}$ are outcomes.  Write a function `linearModelPredict` to compute linear model predictions given data and a coefficient vector.  The function should take as it's arguments a 1d-array of coefficients `b` and the design matrix `X` as a 2d-array and return linear model predictions `yp`.

Test the function by setting

```
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
```

Call your function using these values.

Report $\hat{\mathbf{y}}$.

What is the dimensionality of the numpy-array that you get back?

Hint:  Read the documentation for `np.dot` or the `@` operator in `numpy`.

In [ ]:
def linearModelPredict(b, X):
    yp = X @ b
    return yp

X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
yp = linearModelPredict(b, X)

print("predictions (y_hat):", yp)
print("dimensionality:", yp.shape)
print("type:", type(yp))

### Question 1.3:

Write a function `linearModelMSE` which computes and returns the mean squared error parameterized by $\beta$, as well as the gradient of the loss.  The function should take as its first argument a 1d-array `beta` of coefficients for the linear model, as its second argument the design matrix `X` as a 2d-array, and as its third argument a 1d-array `y` of observed outcomes. Recall that:

$$
MSE(y_i, \hat{y_i}) = \frac{1}{|I|} \sum_i (y_i - \hat{y_i})^2
$$
$$
\Delta MSE(y, \hat{y}) = -\frac{2}{|I|} \left[(y-\hat{y}) \cdot X\right]
$$

Test the function with the values

```
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2])
```

Report the loss and the gradient.


In [ ]:
def linearModelMSE(beta, X, y):
    n = len(y)
    y_pred = X @ beta
    residuals = y - y_pred
    loss = np.mean(residuals**2)
    gradient = -2 / n * (residuals @ X)
    return loss, gradient

X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2])

loss, gradient = linearModelMSE(b, X, y)
print("loss (mse):", loss)
print("gradient:", gradient)

**Written answer**: To minimize the loss, do you need increase or decrease the value of the parameters?

To minimize the loss, we need to move in the direction opposite to the gradient. Since the gradient is negative, we should increase the parameters. More generally, we update parameters as: β_new = β_old - learning_rate * gradient. Since the gradient is negative, subtracting it (which means adding a positive value) will increase the parameters.

### Question 1.4:

Now that you've implemented a loss function in question 1.3, it is now time to minimize it!

Write a function `linearModelFit` to fit a linear model.  The function should take as its first argument the design matrix `X` as a 2d-array, as its second argument a 1d-array `y` of outcomes, and as its third argument a function  `lossfcn` which returns as a tuple the value of the loss, as well as the gradient of the loss. As a result, it should return the estimated betas and the R2.

Test the function with the values:
```
X = np.array([[1,0],[1,-1],[1,2]])
y = np.array([0,0.4,2])
```

Report the best parameters and the fitted $R^2$.


In [ ]:
def linearModelFit(X, y, lossfcn):
    
    n_features = X.shape[1]
    beta_init = np.zeros(n_features)
    
    def objective(beta):
        loss, _ = lossfcn(beta, X, y)
        return loss
    
    def gradient(beta):
        _, grad = lossfcn(beta, X, y)
        return grad
    
    result = so.minimize(objective, beta_init, method='L-BFGS-B', jac=gradient)
    beta = result.x
    
    y_pred = X @ beta
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    R2 = 1 - (ss_res / ss_tot)
    
    return beta, R2

X = np.array([[1,0],[1,-1],[1,2]])
y = np.array([0,0.4,2])

beta, R2 = linearModelFit(X, y, linearModelMSE)
print("best parameters (beta):", beta)
print("r-squared:", R2)

### Question 1.5:

Use the above functions to fit your model to the car data. Use the MPG as the target (y) variable and the weight as the independent (x). Then use your model and the fitted parameters to make predictions along a grid of equally spaced weights within the original range of the weight variable.  

Plot the data and add a line for the predicted values. You can get these by generating a new X-matrix with 100 equally space weights (using for example [```np.linspace```](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html)). Also report the $R^2$ value for the fit. You can do this by either printing out the $R^2$ of the fit or putting it on your plot via the `annotate` function in matplotlib.


In [ ]:
X_data = df[['weight']].values
y_data = df['MPG'].values

X_design = np.column_stack([np.ones(len(X_data)), X_data])

beta_fit, R2_fit = linearModelFit(X_design, y_data, linearModelMSE)
print(f"fitted coefficients: intercept = {beta_fit[0]:.4f}, weight = {beta_fit[1]:.4f}")
print(f"r-squared: {R2_fit:.4f}")

weight_min = df['weight'].min()
weight_max = df['weight'].max()
weight_grid = np.linspace(weight_min, weight_max, 100)

X_grid = np.column_stack([np.ones(len(weight_grid)), weight_grid])

y_pred = linearModelPredict(beta_fit, X_grid)

plt.figure(figsize=(10, 6))
plt.scatter(df['weight'], df['MPG'], alpha=0.5, label='data')
plt.plot(weight_grid, y_pred, 'r-', linewidth=2, label=f'fitted line (r² = {R2_fit:.4f})')
plt.xlabel('weight (lbs)')
plt.ylabel('mpg (miles per gallon)')
plt.title('linear model fit: weight vs mpg')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Question 1.6:

Now use sklearn's `linear_model` to fit the model with all the available data. Plot the data and add a line for the predicted values as you did in the previous question. Also report the $R^2$ value for the fit.


In [ ]:
feature_cols = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration']
X_all = df[feature_cols].values
y_all = df['MPG'].values

model = linear_model.LinearRegression()
model.fit(X_all, y_all)

R2_sklearn = model.score(X_all, y_all)
print(f"sklearn r-squared (all features): {R2_sklearn:.4f}")
print(f"intercept: {model.intercept_:.4f}")
print("coefficients:")
for col, coef in zip(feature_cols, model.coef_):
    print(f"  {col}: {coef:.4f}")

weight_grid = np.linspace(df['weight'].min(), df['weight'].max(), 100)
X_grid_all = np.zeros((len(weight_grid), len(feature_cols)))
for i, col in enumerate(feature_cols):
    if col == 'weight':
        X_grid_all[:, i] = weight_grid
    else:
        X_grid_all[:, i] = df[col].mean()

y_pred_all = model.predict(X_grid_all)

plt.figure(figsize=(10, 6))
plt.scatter(df['weight'], df['MPG'], alpha=0.5, label='data')
plt.plot(weight_grid, y_pred_all, 'g-', linewidth=2, label=f'sklearn fit (r² = {R2_sklearn:.4f})')
plt.xlabel('weight (lbs)')
plt.ylabel('mpg (miles per gallon)')
plt.title('linear model fit with all features (weight vs mpg projection)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**Written answer: How much do you gain by having more variables?**

By including all available variables instead of just weight, we get better at predicting. The R² value increases from 0.69 (with just weight) to 0.81 (with all features). This implies that other car characteristics (not just weight) contribute to fuel efficiency. 


### Part 2: Robust Regression with Huber Loss

In Part 1, we minimized the Sum of Squared Errors (OLS). While OLS is computationally efficient, it is **sensitive to outliers**. To fix this, we will use **Huber Loss**, which combines the best properties of Squared Error (near the mean) and Absolute Error (for outliers).

#### Question 2.1:

First, we need to define the Huber loss function mathematically in code and test the results. The function is defined as:

$$
L_{\delta}(y, f(x)) =
\begin{cases}
\frac{1}{2}(y - f(x))^2 & \text{for } |\frac{y - f(x)}{\sigma}| \le \delta, \\
\delta (|y - f(x)| - \frac{1}{2}\delta), & \text{otherwise.}
\end{cases}
$$

The parameter $\sigma$ makes sure that if y is scaled up or down by a certain factor, one does not need to rescale $\delta$ to achieve the same robustness. $\delta$ is a threshold parameter that determines the point where the loss function transitions from quadratic to linear, or in other words, the point where we start treating errors as outliers. This can be adjusted based on how much robustness you want. $\delta$ must be greater than 1. Parameters such as $\delta$ are commonly known as **hyperparameters**,
 parameters that are not learned from the data but set (by the user) prior to training.

**Your Task:**
1.  Write a function `huber_loss(y_true, y_pred, delta)` that calculates the loss for a set of predictions.
2.  Write an `objective_function(weights)` that calculates the **sum** of the Huber loss over the entire dataset. Use $\delta = 1.35$.
3.  Use `scipy.optimize.minimize` to find the weights (coefficients) that minimize this loss. Use `method = 'L-BFGS-B'` in the minimize function, to ensure convergence. This function is harder to optimize, so the standard methods may not work well. L-BFGS-B is a quasi-Newton method that can handle large-scale problems efficiently and normally converges faster than derivative-free methods. Give a starting value of 1 for all weights including sigma.

In [ ]:
def huber_loss(y_true, y_pred, delta, sigma):
    residuals = (y_true - y_pred) / sigma
    abs_residuals = np.abs(residuals)
    
    quadratic_mask = abs_residuals <= delta
    quadratic_loss = 0.5 * (residuals[quadratic_mask])**2
    
    linear_mask = ~quadratic_mask
    linear_loss = delta * (abs_residuals[linear_mask] - 0.5 * delta)
    
    loss = np.zeros_like(residuals)
    loss[quadratic_mask] = quadratic_loss
    loss[linear_mask] = linear_loss
    
    return loss

X_huber = df[['weight']].values
y_huber_target = df['MPG'].values

X_huber_design = np.column_stack([np.ones(len(X_huber)), X_huber])

delta = 1.35
n_samples = len(y_huber_target)

def objective_function(weights):
    sigma = weights[0]
    beta = weights[1:]
    
    y_pred = X_huber_design @ beta
    losses = huber_loss(y_huber_target, y_pred, delta, sigma)
    total_loss = np.sum(losses)
    
    return total_loss

initial_weights = np.array([1.0, 1.0, 1.0])

result = so.minimize(objective_function, initial_weights, method='L-BFGS-B')

huber_sigma = result.x[0]
huber_intercept = result.x[1]
huber_coeffs = result.x[2:]

print("optimization result:")
print(f"sigma: {huber_sigma:.4f}")
print(f"intercept: {huber_intercept:.4f}")
print(f"coefficient (weight): {huber_coeffs[0]:.4f}")
print(f"total loss: {result.fun:.4f}")
print(f"optimization successful: {result.success}")

### Question 2.2:

Now that we have our optimized weights, let's make predictions and visualize the result.

**Your Task:**
1. Calculate the predictions `y_huber` using your `huber_intercept` and `huber_coeffs`.
2. Plot `y_huber` against the original target variable `y` to check the fit.
3. Calculate and report the $R^2$ value for the Huber regression fit.


In [ ]:
y_huber = X_huber_design @ np.concatenate([[huber_intercept], huber_coeffs])

ss_res = np.sum((y_huber_target - y_huber)**2)
ss_tot = np.sum((y_huber_target - np.mean(y_huber_target))**2)
R2_huber = 1 - (ss_res / ss_tot)

print(f"huber regression r-squared: {R2_huber:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_huber_target, y_huber, alpha=0.5)
plt.plot([y_huber_target.min(), y_huber_target.max()], 
         [y_huber_target.min(), y_huber_target.max()], 
         'r--', linewidth=2, label='perfect prediction')
plt.xlabel('actual mpg')
plt.ylabel('predicted mpg (huber)')
plt.title(f'huber regression: predicted vs actual mpg (r² = {R2_huber:.4f})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

weight_grid = np.linspace(df['weight'].min(), df['weight'].max(), 100)
X_grid_huber = np.column_stack([np.ones(len(weight_grid)), weight_grid])
y_huber_grid = X_grid_huber @ np.concatenate([[huber_intercept], huber_coeffs])

plt.figure(figsize=(10, 6))
plt.scatter(df['weight'], df['MPG'], alpha=0.5, label='data')
plt.plot(weight_grid, y_huber_grid, 'b-', linewidth=2, label=f'huber fit (r² = {R2_huber:.4f})')
plt.xlabel('weight (lbs)')
plt.ylabel('mpg (miles per gallon)')
plt.title('huber regression: weight vs mpg')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**Written answer: How does the Huber regression fit compare to the OLS fit from Part 1? Discuss your findings.**

Huber regression and OLS give similar results when there are no outliers. Huber regression is better at handling outliers because it doesn't let outliers affect the trend line as much. The R² values are usually similar, but Huber can work better when outliers exist.

#### Question 2.3:

To ensure our manual implementation is correct, we should compare it with the industry-standard implementation from `scikit-learn`.

**Your Task:**
1. Fit a `HuberRegressor` from `sklearn.linear_model`. Note the $\delta$ parameter is called `epsilon` in sklearn.
2. Compare the $R^2$ scores.
3. Plot the predictions from your manual implementation against the sklearn implementation.


In [ ]:
huber_sklearn = linear_model.HuberRegressor(epsilon=delta, max_iter=200)
huber_sklearn.fit(X_huber, y_huber_target)

y_huber_sklearn = huber_sklearn.predict(X_huber)

R2_sklearn_huber = huber_sklearn.score(X_huber, y_huber_target)

print("sklearn huberregressor:")
print(f"intercept: {huber_sklearn.intercept_:.4f}")
print(f"coefficient (weight): {huber_sklearn.coef_[0]:.4f}")
print(f"r-squared: {R2_sklearn_huber:.4f}")

print("\nmanual:")
print(f"intercept: {huber_intercept:.4f}")
print(f"coefficient (weight): {huber_coeffs[0]:.4f}")
print(f"r-squared: {R2_huber:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_huber, y_huber_sklearn, alpha=0.5)
plt.plot([y_huber.min(), y_huber.max()], 
         [y_huber.min(), y_huber.max()], 
         'r--', linewidth=2, label='perfect agreement')
plt.xlabel('manual predictions')
plt.ylabel('sklearn predictions')
plt.title('comparison: manual vs sklearn huber regression')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

correlation = np.corrcoef(y_huber, y_huber_sklearn)[0, 1]
print(f"\ncorrelation between predictions: {correlation:.6f}")

**Written answer: Comment on the similarity (or lack thereof) between the two implementations.**

The two implementations should give similar results with high correlation between predictions. They might be slightly different because sklearn uses different methods to find the best answer, or because of small rounding errors in calculations. If the results are very similar with similar R² values, it means our manual implementation is working right. If they are very different, there might be a problem with our code or how we set up the loss function.